In [ ]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

installations:

1. pip install synapseclient

In [ ]:
import os
import pandas as pd
local_metadata_tsv = "proc.tsv"
local_clusters_fld = os.path.join(os.getcwd(),"clusters")
os.makedirs(local_clusters_fld, exist_ok=True)
local_cell_type_annotation_fld = os.path.join(os.getcwd(),"labels")
os.makedirs(local_cell_type_annotation_fld, exist_ok=True)
syn_encode_cell_types_annotation_dataset_id = 'syn34271785' #ENCODE DATASET OF THE ANNOTATION
local_path_to_download = os.path.join(os.getcwd(),"fragment_files")

In [ ]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [ ]:
%run synapse_utils_from_analysis.ipynb
%run tsv_files_utils_from_analysis.ipynb 

In [ ]:
# read the proc.tsv object. Validation that the annoation file is available
local_metadata_file_location = os.path.join(os.getcwd(),local_metadata_tsv)
if os.path.exists(local_metadata_file_location) == False:
    print("please copy your proc.tsv locally {}".format(local_metadata_file_location))
else:
    print("proc.tsv is available locally {}".format(local_metadata_file_location))

In [ ]:
# replace this with your own data annoation
# TODO: create a method that reads from other annoation files
df_encode_cell_types = read_proc_tsv(local_metadata_file_location)


In [ ]:
df_encode_cell_types.head(2)

In [ ]:
len(df_encode_cell_types['atac_dataset'].to_list())

In [ ]:
# load the list of the fragment
local_fragment_files_path = os.path.join(os.getcwd(),"fragment_files","*","*")
print("local_fragment_files_path")
local_fragment_files = !ls $local_fragment_files_path
local_fragment_files = list(local_fragment_files)
local_fragment_files[4]

- check that the files in the fragment files has matching synapse data

In [ ]:
local_files_atac_dataset = [x.split("_")[-1].split(".")[0] for x in local_fragment_files]
local_files_atac_dataset[0:10]

In [ ]:
df_encode_cell_types_atac_dataset = df_encode_cell_types["atac_dataset"].tolist()
df_encode_cell_types_atac_dataset[0:3]

In [ ]:
# atac dataset in both
atac_dataset_to_fragment = list(set(local_files_atac_dataset) & set(df_encode_cell_types_atac_dataset))
atac_dataset_to_fragment[0:6]

In [ ]:
len(atac_dataset_to_fragment)


In [ ]:
local_fragment_files

In [ ]:
local_fragment_files_path_match =[x for x in local_fragment_files if x.split("_")[-1].split(".")[0] in atac_dataset_to_fragment]
local_fragment_files_path_match[0:3]
len(local_fragment_files)


In [ ]:
# this will check for which fragments do we have annotation data
df_encode_atac_dataset_relevant_rows = df_encode_cell_types[df_encode_cell_types['atac_dataset'].isin(atac_dataset_to_fragment)]
df_encode_atac_dataset_relevant_rows.head(2)

In [17]:
%run tsv_files_utils_from_analysis.ipynb
%run synapse_utils_from_analysis.ipynb

# TODO: for others. will the data come from synpase as well?
#local_clusters_fld
for local_fragment_file in local_fragment_files:

    # Dictionary to store the output file handles with names
    output_handles = {}
    print("!!!!!local_fragment_file is {}".format(local_fragment_file))
    file_atac_dataset_id = local_fragment_file.split("_")[-1].split(".")[0]
    print("file_atac_dataset_id is {}".format(file_atac_dataset_id))

    # Retrieve the synpase ID from the proc.tsv dataframe
    syn_folder_name_for_cell_types = df_encode_atac_dataset_relevant_rows[df_encode_atac_dataset_relevant_rows['atac_dataset'] == file_atac_dataset_id]['file'].to_list()[0]
    print("syn_folder_name_for_cell_types in synpase is {}".format(syn_folder_name_for_cell_types))
    # (starting_folder_id, first_subfolder_name, second_subfolder_name):
    # retrieve the location on synpase
    syn_file_id_list = get_file_names_in_second_subfolder(syn_encode_cell_types_annotation_dataset_id, \
                                                          syn_folder_name_for_cell_types, \
                                                          "labels")
    syn_file_id_for_atac_dataset_id = syn_file_id_list[0] # the label's file synID
    print('syn_file_id_for_atac_dataset_id is {}'.format(syn_file_id_for_atac_dataset_id))
    
    # Download the synpase label file:
    # Obtain a pointer and download the data to full_labels_file_path
    # {}_{}_{}".format(dataset_id,file_id,file_name))
    cell_types_annotation_file_path = os.path.join(local_cell_type_annotation_fld,"{}_{}_{}".format(syn_encode_cell_types_annotation_dataset_id, \
                                                                                                    file_atac_dataset_id, \
                                                                                                    syn_file_id_for_atac_dataset_id))

    print("cell_types_annotation_file_path is {}".format(cell_types_annotation_file_path))
    os_listdir = []
    if os.path.exists(cell_types_annotation_file_path):
        os.listdir(cell_types_annotation_file_path)
    # os_listdir = os.listdir(cell_types_annotation_file_path)
    if len(os_listdir) > 0:
        print("this was already downloaded from synapse")
        full_cell_types_annotation_file_path = os.path.join(cell_types_annotation_file_path,os_listdir[0])
    else:
        cell_types_annotation_file_synHandle = syn.get(entity=syn_file_id_for_atac_dataset_id,downloadLocation=cell_types_annotation_file_path) 
        print('cell_types_annotation_file_synHandle is {}'.format(cell_types_annotation_file_synHandle))
        full_cell_types_annotation_file_path = cell_types_annotation_file_synHandle.path
    
    print('full_cell_types_annotation_file_path is {}'.format(full_cell_types_annotation_file_path))
    
#   # Load the labels data to dataframe. has all the barcodes and the cell types relevant for them
    df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_cell_types_annotation_file_path)
    cell_type_id_names_for_atac_dataset = list(set(df_cell_types_for_atac_dataset['cell_type_id'].to_list()))
    print("number of cell_type_id_names_for_atac_dataset is {}".format(len(cell_type_id_names_for_atac_dataset)))
    # List of output text files with corresponding names
    # [("output1.txt", "file_1"), ("output2.txt", "file_2"), ("output3.txt", "file_3")]
    output_tagAlign_files_with_names = [(os.path.join(local_clusters_fld,file_atac_dataset_id,"tagAlign_{}_{}.tsv".format(file_atac_dataset_id,cell_type_name_id)),
                                        cell_type_name_id) for cell_type_name_id in cell_type_id_names_for_atac_dataset]

    tagAlign_exists = [os.path.exists(output_tagAlign_file[0]) for output_tagAlign_file in output_tagAlign_files_with_names]
    print("tagAlign_exists is {}".format(tagAlign_exists))
    if sum(tagAlign_exists) >0:
        print("output_tagAlign_files_with_names {} is at work or was already downloaded. continue".format(output_tagAlign_files_with_names))
        continue # either started by annother process or already was processed
    else:
        print("!!!output_tagAlign_files_with_names {}. open files".format(output_tagAlign_files_with_names))
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            # print("tag_file_path is {}".format(tag_file_path))
            # print("os.path.dirname(tag_file_path) is {}".format(os.path.dirname(tag_file_path)))
            os.makedirs(os.path.dirname(tag_file_path), exist_ok=True)
            output_handles[tag_file_cell_type_name] = open(tag_file_path, "w")


    print("open local_fragment_file {}".format(local_fragment_file))
    with gzip.open(local_fragment_file, "rt") as infile:
        missing_bc = 0
        num_of_lines_written=0
        for line_number, line in enumerate(infile, start=1):
            # # debug
            # if line_number > 10:
            #     continue 

            out_list = split_fragment_line_string(line)
            bc = out_list[-1]
            # print("XXXXXX bc is {}".format(bc))
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
            out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
            # print("XXXXXX out_line_to_print is {}".format(out_line_to_print))
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
            num_of_lines_written +=1
            returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
            # print("XXXXXX returnTagAlign is {}".format(returnTagAlign))
            # write to the relevant cell type file
            bc_exists_in_cell_type_atac_dataset = df_cell_types_for_atac_dataset[df_cell_types_for_atac_dataset['cell_id']==bc]
            # print("bc_exists_in_cell_type_atac_dataset is {}".format(bc_exists_in_cell_type_atac_dataset))
            # print("len(bc_exists_in_cell_type_atac_dataset) is {} for bc {}".format(len(bc_exists_in_cell_type_atac_dataset),bc))
            if len(bc_exists_in_cell_type_atac_dataset) ==1:
                tag_file_cell_type_id = df_cell_types_for_atac_dataset.loc[df_cell_types_for_atac_dataset['cell_id'] == bc, 'cell_type_id'].iloc[0]
                # print("tag_file_cell_type_id is {}".format(tag_file_cell_type_id))
                output_handles[tag_file_cell_type_id].write(returnTagAlign)
            else:
                missing_bc+=1

        # if the file is not closed. data will be missing
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            # print("tag_file_path is {}".format(tag_file_path))
            output_handles[tag_file_cell_type_name].close()            
        print("finished clustering local_fragment_file {} by cell type. for types {}".format(local_fragment_file, cell_type_id_names_for_atac_dataset))
        print("total missing bc are {}".format(missing_bc))

finished clustering local_fragment_file /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119922_ENCSR411MLR.atac.filter.fragments.hg38.tsv.gz/ENCSR411MLR.atac.filter.fragments.hg38.tsv.gz by cell type. for types ['T_lymphocyte', 'Macrophage', 'Vascular_smooth_muscle_cell.1', 'Vascular_smooth_muscle_cell.2', 'Endothelial_cell']
total missing bc are 20174691
!!!!!local_fragment_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119924_ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz/ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz
file_atac_dataset_id is ENCSR052DKH
syn_folder_name_for_cell_types in synpase is EAID_000043
get_file_names parameters are: folder_id syn34271785 and current_depth 0
get_file_names parameters are: folder_id syn46650139 and current_